In [1]:
import numpy as np
import pandas as pd
import pickle
from helper import *

In [2]:
##  Creating the alpha matrix: expenditure share on good i in region r:
##  (VPA + VGA+ S_i VFA)/GDP

In [3]:
username = 'ozaltun'
root = '/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/output/baseview/{}.csv'

vfa =  pd.read_csv(root.format('SF01'), index_col=0, header=1, low_memory=False)

comm_2_num = load_obj('/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/mapping/comm_2_num.pickle')
reg_2_num = load_obj('/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/mapping/reg_2_num.pickle')
prodcomm_2_num = load_obj('/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/mapping/prodcomm_2_num.pickle')
demdcomm_2_num = load_obj('/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/mapping/demdcomm_2_num.pickle')


/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
vfa.drop(['DIR', 'PURCHVALUE'], axis=1, inplace=True)

In [19]:
vfa_np = convertRow2Matrix(vfa, ['DEMD_COMM','PROD_COMM','REG'], [demdcomm_2_num, prodcomm_2_num, reg_2_num])

Number of empty values:  [-1000. -1000. -1000. ... -1000. -1000. -1000.]


In [20]:
## Lets see if we have the right number of zeros: total possible - number in dataset - no_fills
62*58*134  - vfa.groupby(['DEMD_COMM','PROD_COMM','REG']).sum().reset_index().shape[0] - vfa_np[vfa_np<0].shape[0]

0

In [21]:
vfa_np[vfa_np<0] = 0
vfa_np = vfa_np/vfa_np.sum(axis=0).reshape((1, 58, 134)) # Technically it should be SVA after this

In [22]:
vfa_unsklab = vfa_np[57, :, :]
vfa_capital = vfa_np[58, :, :]
vfa_sklab = vfa_np[59, :, :]
vfa_land = vfa_np[60, :, :]
vfa_natlres = vfa_np[61,:,:]

In [23]:
# vfa_unsklab[(vfa_unsklab<0) & (vfa_sklab>0)] = 0

In [24]:
# vfa_sklab[(vfa_sklab<0) & (vfa_unsklab>0)] = 0

In [25]:
vfa_lab = vfa_unsklab + vfa_sklab

In [26]:
vfa_lab[vfa_lab<0]

array([], dtype=float64)

In [27]:
data = [vfa_lab, vfa_capital, vfa_land, vfa_natlres]
title = ['labor', 'capital', 'land', 'natlres']

In [28]:
root_matrix = '/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/matrices/phi_{}.pickle'

for i in range(4):
    save_obj(data[i], root_matrix.format(title[i]))

In [29]:
phi = vfa_np[:57,:,:]

In [30]:
save_obj(phi, root_matrix.format('comm'))

In [31]:
# Note that the phi created above is phi_ikg

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])

In [35]:
phi_sum = phi.sum(axis=0)